In [2]:
!python -V

Python 3.12.7


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

In [6]:
from sklearn.metrics import root_mean_squared_error

In [7]:
import mlflow

In [11]:
mlflow.set_tracking_uri('http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000')

In [12]:
mlflow.get_tracking_uri()

'http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000'

In [13]:
# Test Run
with mlflow.start_run():
    mlflow.log_param('test_param', 123)
    mlflow.log_metric('test_metric', 0.95)

🏃 View run shivering-fly-931 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/0/runs/2ccec37a4c2244cf8c9393064da237f4
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/0


In [14]:
mlflow.set_experiment('nyc_taxi_experiment')

<Experiment: artifact_location='s3://mlflow-artifacts-experiments/1', creation_time=1747163755802, experiment_id='1', last_update_time=1747163755802, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

This function reads in the parquet files for the green taxi trip data. It calculates the duration of a trip in minutes and keeps trips that have a duration of 1 to 60 minutes. It also updates the categorical features of PULocationID and DOLocationID from numbers to string.

In [15]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [16]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [17]:
len(df_train), len(df_val)

(73908, 61921)

In [18]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [19]:
categorical = ['PU_DO'] # ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715206128955

In [22]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Let's try a run with MLFlow training the Lasso linear model.

In [23]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'michelle')

    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lor = Lasso(alpha)
    lor.fit(X_train, y_train)

    y_pred = lor.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    with open('models/lasso.bin', 'wb') as f_out:
        pickle.dump((dv, lor), f_out)
    mlflow.log_artifact(local_path='models/lasso.bin', artifact_path='models_pickle')

🏃 View run orderly-loon-876 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/944095fc445e4963b734776695ac1358
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1


Now try experimenting with training an XGBoost model.

In [24]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/ubuntu/anaconda3/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('developer', 'michelle')
        mlflow.set_tag('model', 'xgboost2')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.24936                                                                  
[1]	validation-rmse:7.84082                                                                  
[2]	validation-rmse:7.21262                                                                  
[3]	validation-rmse:6.93644                                                                  
[4]	validation-rmse:6.80900                                                                  
[5]	validation-rmse:6.73969                                                                  
[6]	validation-rmse:6.70423                                                                  
[7]	validation-rmse:6.67831                                                                  
[8]	validation-rmse:6.66354                                                                  
[9]	validation-rmse:6.64898                                                                  
[10]	validation-rmse:6.63850                                

In [ ]:
mlflow.end_run()

In [ ]:
print('XGBoost Hyperparameter Tuning Complete')

Train the XGBoost model with the best hyperparameters.

In [25]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.19913755185234963,
        'max_depth': 22,
        'min_child_weight': 1.2142790557597472,
        'objective': 'reg:squarederror',
        'reg_alpha': 0.33313132563273024,
        'reg_lambda': 0.008551282616462179,
        'seed': 42,
    }

    mlflow.set_tag('parameters', 'best')
    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[0]	validation-rmse:10.65472
[1]	validation-rmse:9.50354
[2]	validation-rmse:8.66408
[3]	validation-rmse:8.06285
[4]	validation-rmse:7.63648
[5]	validation-rmse:7.33654
[6]	validation-rmse:7.12703
[7]	validation-rmse:6.98073
[8]	validation-rmse:6.87403
[9]	validation-rmse:6.79659
[10]	validation-rmse:6.73854
[11]	validation-rmse:6.69727
[12]	validation-rmse:6.66759
[13]	validation-rmse:6.64353
[14]	validation-rmse:6.62729
[15]	validation-rmse:6.61298
[16]	validation-rmse:6.60008
[17]	validation-rmse:6.59017
[18]	validation-rmse:6.58302
[19]	validation-rmse:6.57713
[20]	validation-rmse:6.57268
[21]	validation-rmse:6.56978
[22]	validation-rmse:6.56774
[23]	validation-rmse:6.56728
[24]	validation-rmse:6.56577
[25]	validation-rmse:6.56391
[26]	validation-rmse:6.56102
[27]	validation-rmse:6.55979
[28]	validation-rmse:6.55752
[29]	validation-rmse:6.55599
[30]	validation-rmse:6.55399
[31]	validation-rmse:6.55218
[32]	validation-rmse:6.55025
[33]	validation-rmse:6.54883
[34]	validation-rmse:6.

/home/ubuntu/anaconda3/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [19:10:37] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/19 19:10:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rogue-shark-691 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/ae2964d21a614e2a8017afc9b5b5c09f
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1


Compare to other models.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():
        mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
        mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')
        mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('rmse', rmse)

🏃 View run delicate-gnat-156 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/ded287ebf8e9452390da27a86382ea0b
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1
🏃 View run resilient-crow-66 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/bf8a0f11ac8f417b9b2fe57b700494b3
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1


🏃 View run delicate-gnat-156 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/ded287ebf8e9452390da27a86382ea0b
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1
🏃 View run resilient-crow-66 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/bf8a0f11ac8f417b9b2fe57b700494b3
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1
🏃 View run brawny-dolphin-27 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/49166d9f43f94108b913d154eb664898
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1
/home/ubuntu/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
🏃 View run honorable-shrike-60 at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/44aeda4a4a024ca69966fe566e320e2b
🧪 View experiment at: http://ec2-3-16-69-43.us-east-2.compute.amazonaws.com:5000/#/experiments/1